In [ ]:
import numpy as np
import pandas as pd  
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
import pyarrow.feather as feather
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
!pip install shap
import shap
!pip install plotly
import plotly.express as px
# Suppress warnings 
# (sometimes you might want to ignore warnings, that's how you can achieve this)
import warnings
warnings.filterwarnings('ignore')
#! brew install graphviz
RSEED= 42

In [ ]:
export_df = feather.read_feather("../data/cleaned_data.feather")

In [ ]:
# Function to split the dataset 
def splitdataset(df):

    y=export_df["interesting_message"]
    X=export_df.drop("interesting_message",axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X,y, stratify = y, test_size = 0.2,random_state = 42)
    return X_train, X_test, y_train, y_test

In [ ]:
#param_tree = {'max_leaf_nodes':[20,50,100,500],
#    'criterion': ['gini', 'entropy'],
#   'max_depth': [2, 5, 10, 50],
#    'min_samples_leaf':[50,200,500,1000]

In [ ]:
# Function to perform training with MSE. 
def train_tree(X_train, y_train,X_test): 
    # Creating the classifier object 
    reg_tree = DecisionTreeClassifier(class_weight="balanced")
    param_tree = {'max_leaf_nodes':[500],
    'criterion': ['gini', 'entropy'],
    'max_depth': [50],
    'min_samples_leaf':[50]
    }

    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
    grid_tree = GridSearchCV(reg_tree, param_grid=param_tree, cv=cv,scoring='balanced_accuracy', n_jobs=-1, verbose=10)
    grid_tree.fit(X_train, y_train)
    print('Best score:\n{:.2f}'.format(grid_tree.best_score_))
    print("Best parameters:\n{}".format(grid_tree.best_params_))
    print("Best model_tree:\n{}".format(grid_tree.best_estimator_))
    best_model_tree = grid_tree.best_estimator_
    shap_values = shap.TreeExplainer(best_model_tree).shap_values(X_test)
    #X_test.iloc[:10000,:]
    global_importances = np.abs(shap_values).mean(0)[:-1]

    explainer = shap.TreeExplainer(best_model_tree)
    shap_values2 = explainer.shap_values(X_test.iloc[:,:])
    shap.force_plot(explainer.expected_value, shap_values2[0, :], X.iloc[0, :])

    return best_model_tree,global_importances

In [ ]:
# Function to make predictions 
def prediction(X_test, reg_tree): 
    y_pred = reg_tree.predict(X_test)
    return y_pred 

In [ ]:
# Function to calculate accuracy 
def class_metrics(y_test, y_pred):
     
    accuracy = balanced_accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred).round()
    print("Predicted values:\n", y_pred) 
    print("Confusion Matrix: \n", cm) 
    print("Balanced Accuracy: %.4f%%" % (accuracy * 100.0))
    print("Report : \n", classification_report(y_test, y_pred))

    return cm, accuracy

In [ ]:
def plot_importance2 (global_importances):
    #make a bar chart that shows the global importance of the top 20 features
    inds = np.argsort(-global_importances)
    y_pos = np.arange(20)
    inds2 = np.flip(inds[:20], 0)
    f=plt.figure(figsize=(5,10))
    f.barh(y_pos, global_importances[inds2], align='center', color="#1E88E5")
    f.yticks(y_pos, fontsize=13)
    f.gca().set_yticklabels(train.columns[inds2])
    f.xlabel('mean abs. SHAP value (impact on model output)', fontsize=13)
    f.gca().xaxis.set_ticks_position('bottom')
    f.gca().yaxis.set_ticks_position('none')
    f.gca().spines['right'].set_visible(False)
    f.gca().spines['top'].set_visible(False)

def plot_importance (global_importances):
    inds = np.argsort(-global_importances)
    y_pos = np.arange(20)
    inds2 = np.flip(inds[:20], 0)
    fig = px.box(global_importances)
    fig.show()

In [ ]:
# Driver code 
def main(): 
    # Building Phase 
    X_train, X_test, y_train, y_test = splitdataset(export_df) 
    basemodel,importance = train_tree(X_train, y_train,X_test)
    print(f'Decision tree has {basemodel.tree_.node_count} nodes with maximum depth {basemodel.tree_.max_depth}.')
    # Operational Phase 
    print("-----"*15)
    print("Results:\n")
    # Prediction
    y_pred = prediction(X_test, basemodel) 
    cm,accuracy= class_metrics(y_test, y_pred)
    return basemodel,y_pred,cm,accuracy,importance

In [ ]:
basemodel,y_pred,cm,accuracy,importance=main()

In [ ]:
plot_importance(importance)

In [ ]:
from sklearn.tree import export_graphviz  
features=['lenght', 'emoji_size', 'slang_char', 'slang_verb', 'slang_pron',
       'slang_adp', 'slang_noun', 'slang_num', 'slang_punt', 'slang_det',
       'info_char', 'info_words', 'info_verb', 'info_pron', 'info_adp',
       'info_noun', 'info_num', 'info_punt', 'info_det', 'cause_BrokenVehicle',
       'cause_COVID19', 'cause_Counterflow', 'cause_CycleRide',
       'cause_Demonstration', 'cause_EmergencyServices', 'cause_Event',
       'cause_Explosion', 'cause_FallenTree', 'cause_Fire', 'cause_Flood',
       'cause_GasLeak', 'cause_HeavyTraffic', 'cause_Incident',
       'cause_Landslide', 'cause_Leak', 'cause_Maintenance', 'cause_March',
       'cause_Overturn', 'cause_Pilgrimage', 'cause_ProtestCamp', 'cause_Rain',
       'cause_Reopening', 'cause_Sinkhole', 'cause_StreetWorks',
       'cause_VehicularAccident', 'cause_Waterlogging',
       'effect_CirculationRestored', 'effect_CirculationShutdown',
       'effect_Delays', 'effect_Evacuation', 'effect_FullCapacity',
       'effect_HighWaitingTime', 'effect_InterimService',
       'effect_LaneReduction', 'effect_RouteDetour', 'effect_SecuritySpeed',
       'effect_SuspensionOfService', 'effect_TrafficImpact']
# export the decision tree to a tree.dot file 
#for visualizing the plot easily anywhere 
export_graphviz(basemodel, out_file ='tree_class.dot', 
               feature_names =features) 

In [ ]:
! dot -Tpng tree_class.dot > tree_class.png

In [ ]:
f = open('ml-log.txt', 'a')
f.write('Base model: Decision Tree\n Predicted values:\n {}\n Confusion Matrix:\n {}\n Balanced Accuracy:\n {} \n Model: {}'.format(y_pred,cm,accuracy,basemodel))
f.close()